#  SVM Primal Form

#### Support Vector Machines (SVM) is a popular machine learning algorithm used for classification and regression analysis. In this implementation, we will be using SVM for binary classification in the primal form.

#### The primal form of SVM involves finding the optimal hyperplane that separates the two classes of data. The hyperplane is defined by a weight vector `w` and a bias term `b`. The goal is to find the values of `w` and `b` that maximize the margin between the two classes of data.

#### To find the optimal values of `w` and `b`, we can use convex optimization. We can define an objective function that maximizes the margin subject to the constraint that all data points are correctly classified. This can be formulated as a quadratic optimization problem, which can be solved using a convex optimization solver such as CVXPY.

The steps taken in this implementation are as follows:

1. Import the necessary libraries: We import the CVXPY library for convex optimization, and the Pandas and NumPy libraries for data manipulation.

2. Load the training and test datasets: We load the training and test datasets from CSV files using the Pandas library. We extract the input features and target labels from the datasets.

3. Preprocess the data: We preprocess the data by replacing the target labels with -1 and 1, and normalizing the input features.

4. Define the SVM model: We define the SVM model using CVXPY. We define the weight vector `w` and bias term `b` as variables, and define the objective function and constraints.

5. Train the SVM model: We solve the optimization problem using CVXPY to find the optimal values of `w` and `b`.

6. Evaluate the SVM model: We evaluate the SVM model on the test dataset by computing the accuracy and confusion matrix.


Implementation of training and testing of soft-margin Linear Support Vector Machine in its primal form, that is,

$$\min_{\mathbf{w},b,\{\xi_i\}} \frac{1}{2} \|\mathbf{w}\|_2^2 + \frac{C}{N} \sum_{i=1}^N \xi_i \nonumber \\ s.t.~~ y_i (\mathbf{w}^\top \mathbf{x}_i + b) \ge 1 - \xi_i, ~~\forall i \nonumber \\ \xi_i \ge 0 \nonumber$$
Using CVXPY 

In [2]:
import cvxpy as cp # convex optimization package
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

# get training dataset
train = "train.csv"
df = pd.read_csv(train, header=None) # read csv file
X_train = df[:2000].iloc[:, 1:].to_numpy() # get features (2000 samples) 
Y_train = df[:2000].iloc[:, 0].replace(0, -1).to_numpy() # get labels (2000 samples) change 0 to -1

# get test dataset
test = "test.csv" 
df = pd.read_csv(test, header=None) 
X_test = df.iloc[:1000, 1:].to_numpy() 
Y_test = df.iloc[:1000, 0].replace(0, -1).to_numpy()


In [3]:
# train linear svm in primal form
def svm_train_primal(data_train, label_train, regularisation_para_c):
    X, Y = data_train, label_train # X is the data matrix, Y is the label matrix
    n_samples, m_features = np.shape(X) # n_samples is the number of samples, m_features is the number of features
    
    W_value = 0
    b_value = 0
    slack_var_value = 0

# ====================== CODE HERE ======================  
    # Reference used for SVM Primal <https://ai538393399.wordpress.com/2020/11/25/svm-using-cvxpy/>
    W = cp.Variable(m_features) # Weight Matrix
    b = cp.Variable() # Also known as gamma term but tis the b value from above equation
    slack_var = cp.Variable(n_samples) # Psi fromm the above equation  
    objective = cp.Minimize(0.5 * cp.sum_squares(W) + 
                            (regularisation_para_c/n_samples) * 
                            cp.sum(slack_var)) # Objective function from above equation
    constraints = [cp.multiply(Y,  X @ W  + b) - 1 + slack_var >= 0, 
                   slack_var >= 0] # Constraints from above equation
    prob = cp.Problem(objective, constraints) # Problem to be solved
    prob.solve()
    W_value = W.value
    b_value = b.value
    slack_var_value = slack_var.value
# ================================================================

    return [W_value, b_value, slack_var_value]

# train primal model
c = 100 # regularisation parameter
model_primal = svm_train_primal(X_train, Y_train, c) # train primal model

# output svm primal form solutions
W = model_primal[0] # W is the weight matrix
b = model_primal[1] # b is the bias term

print('Please copy the folowing result to Question 1 "sum(W) = ')
print(np.round(np.sum(W),2))
print('Please copy the folowing result to Question 1 "b = "')
print(np.round(b,2))

Please copy the folowing result to Question 1 "sum(W) = 
0.79
Please copy the folowing result to Question 1 "b = "
1.75


In [4]:
# predict accuracy of svm model on test dataset
def svm_predict(data_test, label_test, svm_model):
    
    acc = 0.0
    
# ======================CODE HERE ======================  
    X = data_test # Here we load the data
    Y = label_test # Here we load the label
    W = svm_model[0] # Here we load the trained parameters
    b = svm_model[1] # Here we load the trained parameters
    Y_pred = np.sign(X @ W + b) # Here we ompute the predicted labels
    acc = np.mean(Y_pred == Y) # Here we compute the accuracy
# ==========================================================

    return acc

# predict and output primal accuracy
accuracy = svm_predict(X_test, Y_test, model_primal)
print('Please copy the folowing result line to Question 1 "Accuracy = )"')
print(np.round(accuracy,2))

Please copy the folowing result line to Question 1 "Accuracy = )"
0.96
